# Image Classification

In [1]:
import os
import matplotlib.pyplot as olt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms,models,datasets
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

### Import Training and Validation Image

In [2]:
data_dir='./flower_data/'
train_dir=data_dir+'/train'
valid_dir=data_dir+'/valid'

### Data Pre-Processsing

In [3]:
# Data Augmentation
data_transforms={
    'train':transforms.Compose([transforms.RandomRotation(45),                  # Random Rotation between -45 to +45 degree
    transforms.CenterCrop(224),                                                 # Cut from Center, VGG, Resnet
    transforms.RandomHorizontalFlip(p=0.5),                                     # Random Horizontal Flip
    transforms.RandomVerticalFlip(p=0.5),                                       # Random Vertical Flip
    transforms.ColorJitter(brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1), # brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1
    transforms.RandomGrayscale(p=0.025),                                        # Probability to get Grayscale
    transforms.ToTensor(),
    transforms.Normalize([0.458,0.456,0.406],[0.229,0.224,0.225])               # Mean and Var, make sure our data fit other model like VGG Resnet
    ]),
    'valid':transforms.Compose([transforms.Resize(256),
    transforms.CenterCrop(224),                                                 # Cut from Center
    transforms.ToTensor(),
    transforms.Normalize([0.458,0.456,0.406],[0.229,0.224,0.225])               # Mean and Var
    ])
}

In [4]:
batch_size=8
image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train', 'valid']} # root=os.path.join(data_dir,x), transforms=data_transforms[x], in ['train', 'valid']
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=batch_size,shuffle=True) for x in ['train', 'valid']}
dataset_sizes={x:len(image_datasets[x]) for x in ['train', 'valid']}
class_name=image_datasets['train'].classes

with open ('cat_to_name.json','r') as f: # Read name of flower
    cat_to_name=json.load(f)


In [5]:
def im_convert(tensor):
    '''Convert tensor data type back to noraml image data type'''
    image = tensor.to('cpu').clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229,0.224,0.225))+np.array((0.458,0.456,0.406))
    image = image.clip(0,1)
    return image

### Load model from models, and use trained weights and bias

In [6]:
model_name='resnet' # Can choose from ['resnet','alexnet','vgg',squeezenet','densenet','inception']
feature_extract=True

train_on_gpu=torch.cuda.is_available()

# Check for GPU if available use GPU
if not train_on_gpu:
    print('CUDA is not available. Training on CPU')
else:
    print('CUDA is available. Training on GPU')

device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

CUDA is not available. Training on CPU


In [8]:
# Set the layers that don't need modified
def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad=False

In [9]:
model_ft=models.resnet152() # Load Resnet 152
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Example from pytorch

In [10]:
def initialize_model(model_name,num_classes, feature_extract,use_pretrained=True):
    # Choose suitable model, There are some different for different model
    model_ft=None
    input_size=0

    if model_name=='resnet':
        '''Resnet152'''
        model_ft=models.resnet152(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        num_ftrs=model_ft.fc.in_features
        model_ft.fc=nn.Sequential(nn.Linear(num_ftrs,102),
                                  nn.LogSoftmax(dim=1))
        input_size=224
    elif model_name=='alexnet':
        '''Alexnet'''
        model_ft=models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        num_ftrs=model_ft.classifier[6].in_features
        model_ft.classifier[6]=nn.Linear(num_ftrs,num_classes)
        input_size=224
    elif model_name=='vgg':
        '''VGG11_bn'''
        model_ft=models.vgg16(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        num_ftrs=model_ft.classifier[6].in_features
        model_ft.classifier[6]=nn.Linear(num_ftrs,num_classes)
        input_size=224
    elif model_name=='squeezenet':
        '''Squeezenet'''
        model_ft=models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        num_ftrs.classifier[1]=nn.Conv2d(512,num_classes,kernel_size=(1,1),stride=(1,1))
        model_ft.num_classes=num_classes
        input_size=224
    elif model_name=='densenet':
        '''Densenet'''
        model_ft=models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        num_ftrs=model_ft.classifier.in_features
        model_ft.classifier=nn.Linear(num_ftrs,num_classes)
        input_size=224
    elif model_name=='inception':
        '''Inception v3
            Be careful, expects (299,299) sized images and has auxiliary output
        '''
        model_ft=models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft,feature_extract)
        # Handle the auxilary net
        num_ftrs=model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc=nn.Linear(num_ftrs,num_classes)
        # Handle the primary net
        num_ftrs=model_ft.fc.in_features
        model_ft.fc=nn.Linear(num_ftrs,num_classes)
        input_size=299

    else:
        print('Invalid model name, exiting')
        exit()

    return model_ft,input_size

In [12]:
model_ft,input_size=initialize_model(model_name,102,feature_extract,use_pretrained=True)

model_ft=model_ft.to(device)

# Save model
filename='checkpoint.pth'

# Whether train all the layer
params_to_update=model_ft.parameters()
print('Params to lean:')
if feature_extract:
    params_to_update=[]
    for name,param in model_ft.named_parameters():
        if param.requires_grad==True:
            params_to_update.append(param)
            print('\t',name)
else:
    for name,param, in model_ft.named_parameters():
        if param.requires_grad==True:
            print('\t',name)

Params to lean:
	 fc.0.weight
	 fc.0.bias


In [13]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Optimizer Settings

In [ ]:
optimizer_ft=optim.Adam(params_to_update,lr=1e-2)
scheduler=optim.lr_scheduler.StepLR(optimizer_ft,step_size=7,gamma=0.1) # Learning Rate for every 7 epoch decrease to 1/10
# In last layer we have LogSoftmax(), SO we can not use nn.CrossEntropyLoss() any more, nn.CrossEntropyLoss equal to LogSoftmax() plus nn.NLLLoss.
criterion=nn.NLLLoss()

### Training module

In [14]:
def train_model (model,dataloaders,criterion,optimizer,num_epochs=25,is_inception=False,filename=filename):
    

[Parameter containing:
 tensor([[ 5.4514e-03, -1.2473e-02,  1.9192e-02,  ...,  8.5399e-03,
           1.4887e-02,  6.0777e-04],
         [ 2.0857e-02,  9.1214e-03, -1.7113e-02,  ..., -5.8155e-03,
           7.2952e-03, -3.8019e-03],
         [-2.0800e-02, -2.2805e-04,  7.1330e-03,  ..., -5.8194e-03,
          -1.5258e-02,  1.3513e-02],
         ...,
         [-1.8719e-02, -1.8372e-02,  1.1499e-02,  ..., -1.6009e-02,
           1.5106e-02, -1.2466e-02],
         [-8.9938e-05, -1.5362e-02,  1.3961e-02,  ...,  1.1245e-02,
          -1.7975e-02, -1.0640e-02],
         [ 1.1893e-02, -1.7581e-02, -7.3746e-03,  ..., -6.1103e-03,
           1.5088e-02, -1.2802e-02]], requires_grad=True),
 Parameter containing:
 tensor([ 6.9779e-03,  1.9434e-04,  6.7298e-03,  4.1523e-03, -1.5877e-02,
          2.0502e-02,  1.2797e-02, -2.0381e-02,  1.9014e-02,  6.8966e-03,
         -3.4602e-03,  8.1934e-03,  2.1756e-03,  7.1440e-03,  1.8374e-03,
         -1.1100e-02,  3.6549e-03, -1.9054e-02, -1.0108e-02, -1.08